# Inversion benchmark - prior covariance

We want to create a sparse covariance matrix where each node in the domain is positively correlated with its adjacent node. A *Gaussian covariance function* should give us what we want. Ideally, we would create a covariance matrix using the neighbours algorithm baked into the `conduction` package. However, it seems like a more practical way forward is to use `scipy.ndimage.gaussian_filter` to smooth the transitions to different lithologies.

In [ ]:
import numpy as np
from scipy import sparse
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline
import conduction

Use Gaussian filter to smooth field

In [ ]:
def misfit(x, x0, sigma_x0, smooth_fn, *args):
    """ L-norm misfit function"""
    return np.sum(smooth_fn((x - x0)**2/sigma_x0**2, *args))

def misfit_ad(x, x0, sigma_x0, smooth_fn, *args):
    return smooth_fn((2.0*x - 2.0*x0)/sigma_x0**2, *args)

def forward_model(x, *args):
    return misfit(x, x0, sigma_x0, ndimage.gaussian_filter, *args)
def tangent_linear(x, dx, *args):
    cost = misfit(x, x0, sigma_x0, ndimage.gaussian_filter, *args)
    dcdx = misfit_ad(x, x0, sigma_x0, ndimage.gaussian_filter, *args)
    dc = np.sum(dcdx.ravel()*dx.ravel())
    return cost, dc
def adjoint_model(x, *args):
    cost = misfit(x, x0, sigma_x0, ndimage.gaussian_filter, *args)
    grad = misfit_ad(x, x0, sigma_x0, ndimage.gaussian_filter, *args)
    return cost, grad


xmin, xmax = 0., 1.
ymin, ymax = 0., 1.
nx, ny = 10, 10

mesh = conduction.ConductionND((xmin,ymin), (ymin, ymax), (nx,ny), stencil_width=1)
mesh.diffusivity[:] = np.ones(mesh.nn)

In [ ]:
x0 = np.ones(mesh.n)*3.5
sigma_x0 = prior/10.

x = prior.copy() + np.random.randn(*mesh.n)
dx = x*0.01

plt.imshow(x)
plt.colorbar()

In [ ]:
sigma = 1.0

fm0 = forward_model(x, sigma)
fm1 = forward_model(x+dx, sigma)
tl  = tangent_linear(x, dx, sigma)
ad  = adjoint_model(x, sigma)


print("finite difference {}".format(fm1 - fm0))
print("tangent linear {}".format(tl[1]))
print("adjoint model {}".format(ad[1].ravel().dot(dx.ravel())))

In [ ]:
plt.imshow(ad[1])
plt.colorbar()

Use `conduction` package to find neighbouring nodes.

In [ ]:
def misfit(x, x0, sigma_x0, inv_cov):
    """ L-norm misfit function"""
    return np.sum(inv_cov*(x - x0)**2/sigma_x0**2)

def misfit_ad(x, x0, sigma_x0, inv_cov):
    return inv_cov*(2.0*x - 2.0*x0)/sigma_x0**2

def covariance(sigma_x0, x, xprime, L):
    """ Gaussian covariance function """
    return sigma_x0**2 * np.exp(-np.sqrt((x - xprime)**2)/(2*L**2))


In [ ]:
xmin, xmax = 0., 1.
ymin, ymax = 0., 1.
nx, ny = 10, 10

mesh = conduction.ConductionND((xmin,ymin), (ymin, ymax), (nx,ny), stencil_width=2)
mesh.diffusivity[:] = np.ones(mesh.nn)

mat = mesh.construct_matrix()

indptr, indices, values = mat.getValuesCSR()

cov = sparse.csr_matrix((values, indices, indptr)).toarray()

In [ ]:
prior = np.ones(mesh.nn)*3.5
sigma_prior = prior/10.

cov.dot(prior)

In [ ]:
prior = np.ones(mesh.nn)
sigma_prior = prior/2.

neighbours = mesh.find_neighbours(2)

cov = np.zeros((mesh.nn,mesh.nn))

for row in xrange(0, mesh.nn):
    nrow = neighbours[row]
    n = nrow[nrow >= 0]
    i = n[-1]

    ncoord = mesh.coords[n]
    icoord = mesh.coords[i]
    dist = np.linalg.norm(icoord - ncoord, axis=1)

    L = (dist.max() - dist.min()) # length scale
    sigma = np.std(dist)
    
    icov = (sigma_prior[i]/prior[i])**2 * np.exp(-dist**2/(2*L**2))
    icov = 
#     icov = covariance(0.01, 1.0, 1.0, n.size)
    
    cov[row,n] = icov

In [ ]:
plt.imshow(cov)
plt.colorbar()

In [ ]:
invcov = np.linalg.inv(cov)
print cov.dot(prior)

In [ ]:
prior = np.random.randn(*mesh.n)
smooth_prior = cov.dot(prior.ravel()).reshape(mesh.n)

fig, (ax1, ax2) = plt.subplots(1,2,sharey=True, figsize=(10,4))

im1 = ax1.imshow(prior)
im2 = ax2.imshow(smooth_prior)

fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax2)

ax1.set_title('prior')
ax2.set_title('smooth_prior')

In [ ]:
invcov = np.linalg.inv(cov)

plt.imshow(invcov)
plt.colorbar()

$\mathbf{C}^{-1} x_{\mathrm{prior}}$ should at least give us something resembling the prior.

It does not.

In [ ]:
# diagonal covariance - should always return our priors

cov = np.zeros((mesh.nn, mesh.nn))
np.fill_diagonal(cov, 1)

invcov = np.linalg.inv(cov)
invcov.dot(np.ones(mesh.nn))

Use `scipy.ndimage.gaussian_filter` to smooth the field

In [ ]:
prior = np.random.randn(*mesh.n)
sigma_prior = prior/10.

smooth_prior = ndimage.gaussian_filter(prior, 1.)

fig, (ax1, ax2) = plt.subplots(1,2,sharey=True, figsize=(8,4))

ax1.imshow(prior)
ax2.imshow(smooth_prior)

ax1.set_title('prior')
ax2.set_title('smooth_prior')

Ascertain covariance from an ensemble of Gaussian blurs.

In [ ]:
nsim = 10000

gauss = np.empty((nsim, prior.size))

for i in xrange(nsim):
    prior = np.random.randn(*mesh.n)
    smooth_prior = ndimage.gaussian_filter(prior, 0.6)
    gauss[i] = smooth_prior.ravel()

cov = np.cov(gauss.T)
invcov = np.linalg.inv(cov)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,sharey=True, figsize=(10,4))

im1 = ax1.imshow(cov)
im2 = ax2.imshow(invcov)

fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax2)

ax1.set_title('cov')
ax2.set_title('inv cov')

In [ ]:
prior = np.random.random(mesh.nn).reshape(mesh.n)
smooth_prior = cov.dot(prior.ravel()).reshape(mesh.n)

fig, (ax1, ax2) = plt.subplots(1,2,sharey=True, figsize=(10,4))

ax1.imshow(prior)
ax2.imshow(smooth_prior)

fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax2)

ax1.set_title('prior')
ax2.set_title('smooth_prior')

This certainly smooths the variation, but the scale is significantly different!